In [1]:
from simtk import unit
from openmmtools import testsystems, cache, mcmc
from openmmtools.states import ThermodynamicState, SamplerState
import pickle
import mdtraj as md
import numpy as np
import abc
from openmmtools.utils import SubhookedABCMeta, Timer
import copy
import oeommtools.utils as oeommtools
import openeye.oechem as oechem
import random
import math

In [2]:
from simtk import openmm, unit
from simtk.openmm import app

In [3]:
from openmmtools import testsystems

In [128]:
import logging 
logger = logging.getLogger("move")
logger.setLevel(logging.INFO)

Read in THR

In [172]:
with open("/data/chodera/zhangi/perses_benchmark/neq/7/13/13_vacuum.pickle", 'rb') as f:
    htf = pickle.load(f)
    

In [173]:
top = htf._topology_proposal.old_topology


In [7]:
thermodynamic_state = ThermodynamicState(system=htf._topology_proposal.old_system,
...                                          temperature=298*unit.kelvin)


In [8]:
sampler_state = SamplerState(positions=htf.old_positions(htf.hybrid_positions))


Read in alanine dipeptide

In [77]:
test = testsystems.AlanineDipeptideVacuum()


In [78]:
thermodynamic_state = ThermodynamicState(system=test.system,
...                                          temperature=300*unit.kelvin)

In [101]:
test.positions

Quantity(value=array([[ 2.0000010e-01,  1.0000000e-01, -1.3000000e-07],
       [ 2.0000010e-01,  2.0900000e-01,  1.0000000e-08],
       [ 1.4862640e-01,  2.4538490e-01,  8.8982400e-02],
       [ 1.4862590e-01,  2.4538520e-01, -8.8982000e-02],
       [ 3.4274200e-01,  2.6407950e-01, -3.0000000e-07],
       [ 4.3905800e-01,  1.8774060e-01, -6.6000000e-07],
       [ 3.5553754e-01,  3.9696488e-01, -3.1000000e-07],
       [ 2.7331200e-01,  4.5561601e-01, -1.3000000e-07],
       [ 4.8532621e-01,  4.6139253e-01, -4.3000000e-07],
       [ 5.4075960e-01,  4.3155388e-01,  8.8981520e-02],
       [ 5.6613044e-01,  4.2208425e-01, -1.2321480e-01],
       [ 5.1232615e-01,  4.5213630e-01, -2.1312016e-01],
       [ 6.6304840e-01,  4.7189354e-01, -1.2057907e-01],
       [ 5.8085401e-01,  3.1408724e-01, -1.2413850e-01],
       [ 4.7126759e-01,  6.1294185e-01,  1.4000000e-07],
       [ 3.6006445e-01,  6.6527027e-01,  6.2000000e-07],
       [ 5.8460533e-01,  6.8348833e-01,  2.5000000e-07],
       [ 6.73700

In [102]:
sampler_state = SamplerState(positions=test.positions)

In [103]:
sampler_state.positions

Quantity(value=array([[ 2.0000010e-01,  1.0000000e-01, -1.3000000e-07],
       [ 2.0000010e-01,  2.0900000e-01,  1.0000000e-08],
       [ 1.4862640e-01,  2.4538490e-01,  8.8982400e-02],
       [ 1.4862590e-01,  2.4538520e-01, -8.8982000e-02],
       [ 3.4274200e-01,  2.6407950e-01, -3.0000000e-07],
       [ 4.3905800e-01,  1.8774060e-01, -6.6000000e-07],
       [ 3.5553754e-01,  3.9696488e-01, -3.1000000e-07],
       [ 2.7331200e-01,  4.5561601e-01, -1.3000000e-07],
       [ 4.8532621e-01,  4.6139253e-01, -4.3000000e-07],
       [ 5.4075960e-01,  4.3155388e-01,  8.8981520e-02],
       [ 5.6613044e-01,  4.2208425e-01, -1.2321480e-01],
       [ 5.1232615e-01,  4.5213630e-01, -2.1312016e-01],
       [ 6.6304840e-01,  4.7189354e-01, -1.2057907e-01],
       [ 5.8085401e-01,  3.1408724e-01, -1.2413850e-01],
       [ 4.7126759e-01,  6.1294185e-01,  1.4000000e-07],
       [ 3.6006445e-01,  6.6527027e-01,  6.2000000e-07],
       [ 5.8460533e-01,  6.8348833e-01,  2.5000000e-07],
       [ 6.73700

In [8]:
top = test.topology

In [9]:
for atom in top.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ALA)>
<Atom 7 (H) of chain 0 residue 1 (ALA)>
<Atom 8 (CA) of chain 0 residue 1 (ALA)>
<Atom 9 (HA) of chain 0 residue 1 (ALA)>
<Atom 10 (CB) of chain 0 residue 1 (ALA)>
<Atom 11 (HB1) of chain 0 residue 1 (ALA)>
<Atom 12 (HB2) of chain 0 residue 1 (ALA)>
<Atom 13 (HB3) of chain 0 residue 1 (ALA)>
<Atom 14 (C) of chain 0 residue 1 (ALA)>
<Atom 15 (O) of chain 0 residue 1 (ALA)>
<Atom 16 (N) of chain 0 residue 2 (NME)>
<Atom 17 (H) of chain 0 residue 2 (NME)>
<Atom 18 (C) of chain 0 residue 2 (NME)>
<Atom 19 (H1) of chain 0 residue 2 (NME)>
<Atom 20 (H2) of chain 0 residue 2 (NME)>
<Atom 21 (H3) of chain 0 residue 2 (NME)>


Here is tyr

In [175]:
with open("/data/chodera/zhangi/perses_benchmark/neq/7/9/9_vacuum.pickle", 'rb') as f:
    htf = pickle.load(f)
    

In [176]:
top = htf._topology_proposal.old_topology


In [177]:
thermodynamic_state = ThermodynamicState(system=htf._topology_proposal.old_system,
...                                          temperature=298*unit.kelvin)


In [178]:
sampler_state = SamplerState(positions=htf.old_positions(htf.hybrid_positions))


Here is thrombin

In [89]:

from perses.utils.openeye import system_generator_wrapper, createOEMolFromSDF, extractPositionsFromOEMol, OEMol_to_omm_ff

In [90]:
oemol = createOEMolFromSDF("../input/Thrombin_ligands.sdf")

INFO:utils.openeye:molecule 1b                      does not have unique atom names. Generating now...


In [91]:
oemol

<oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x2b46ba2eed20> >

In [92]:
system_generator = system_generator_wrapper([oemol])

DEBUG:perses.forcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


In [93]:
system, positions, topology = OEMol_to_omm_ff(oemol, system_generator)

INFO:openmmforcefields.generators.template_generators:Requested to generate parameters for residue <Residue 0 (1b) of chain 0>
INFO:openmmforcefields.generators.template_generators:Generating a residue template for [H]c1c(c(c(c(c1[H])[H])C([H])([H])[C@]([H])(C(=O)N2[C@](C(C(C2([H])[H])([H])[H])([H])[H])([H])C(=O)N([H])C([H])([H])c3c(c(c(c(c3[H])Cl)[H])[H])[H])[N+]([H])([H])[H])[H])[H]
DEBUG:openmmforcefields.generators.template_generators:Total charge is 1
DEBUG:openmmforcefields.generators.template_generators:Computing AM1-BCC charges...
DEBUG:openmmforcefields.generators.template_generators:Generating a conformer...
DEBUG:openmmforcefields.generators.template_generators:Running antechamber...
DEBUG:openmmforcefields.generators.template_generators:antechamber -i in.mdl -fi mdl -o out.mol2 -fo mol2 -s 0 -at 2 -dr no
DEBUG:openmmforcefields.generators.template_generators:
Welcome to antechamber 19.0: molecular input file processor.


DEBUG:openmmforcefields.generators.template_generator

In [81]:
system

<simtk.openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x2b4628614180> >

In [101]:
thermodynamic_state = ThermodynamicState(system=system,
...                                          temperature=298*unit.kelvin)


In [102]:
sampler_state = SamplerState(positions=positions)


Let's try a hybrid system now

In [213]:
with open("/data/chodera/zhangi/perses_benchmark/neq/7/13/13_vacuum.pickle", 'rb') as f:
    htf = pickle.load(f)
    

In [219]:
top = htf.hybrid_topology


In [215]:
thermodynamic_state = ThermodynamicState(system=htf.hybrid_system,
...                                          temperature=298*unit.kelvin)


In [216]:
sampler_state = SamplerState(positions=htf.hybrid_positions)


In [221]:
for atom in top.atoms:
    print(atom, atom.index)

ACE1-H1 0
ACE1-CH3 1
ACE1-H2 2
ACE1-H3 3
ACE1-C 4
ACE1-O 5
THR2-N 6
THR2-H 7
THR2-CA 8
THR2-HA 9
THR2-CB 10
THR2-C 11
THR2-O 12
THR2-HB 13
THR2-CG2 14
THR2-HG21 15
THR2-HG22 16
THR2-HG23 17
THR2-OG1 18
THR2-HG1 19
THR2-HB2 26
THR2-HB3 27
NME3-N 20
NME3-H 21
NME3-C 22
NME3-H1 23
NME3-H2 24
NME3-H3 25


Below is the first working version of MC dihedral rotation move from BLUES in OpenMMtools

In [8]:
# class MCRotationMove2(mcmc.MetropolizedMove):
#     """A metropolized move that randomly rotate a subset of atoms.
#     Parameters
#     ----------
#     atom_subset : slice or list of int, optional
#         If specified, the move is applied only to those atoms specified by these
#         indices. If None, the move is applied to all atoms (default is None).
#     context_cache : openmmtools.cache.ContextCache, optional
#         The ContextCache to use for Context creation. If None, the global cache
#         openmmtools.cache.global_context_cache is used (default is None).
#     Attributes
#     ----------
#     n_accepted : int
#         The number of proposals accepted.
#     n_proposed : int
#         The total number of attempted moves.
#     atom_subset
#     context_cache
#     See Also
#     --------
#     MetropolizedMove
#     """

#     def __init__(self, topology, chain_index, residue_index, bond_indices, atom_subset=None, context_cache=None):
#         self.n_accepted = 0
#         self.n_proposed = 0
#         self.atom_subset = atom_subset
#         self.context_cache = context_cache
#         self.topology = topology
#         self.chain_index = chain_index
#         self.residue_index = residue_index
#         self.bond_indices = bond_indices

#     def apply(self, thermodynamic_state, sampler_state):
#         """Apply a metropolized move to the sampler state.
#         Total number of acceptances and proposed move are updated.
#         Parameters
#         ----------
#         thermodynamic_state : openmmtools.states.ThermodynamicState
#            The thermodynamic state to use to apply the move.
#         sampler_state : openmmtools.states.SamplerState
#            The initial sampler state to apply the move to. This is modified.
#         """
#         timer = Timer()
#         benchmark_id = 'Applying {}'.format(self.__class__.__name__ )
#         timer.start(benchmark_id)

#         # Check if we have to use the global cache.
#         if self.context_cache is None:
#             context_cache = cache.global_context_cache
#         else:
#             context_cache = self.context_cache

#         # Create context, any integrator works.
#         context, unused_integrator = context_cache.get_context(thermodynamic_state)

#         # Compute initial energy. We don't need to set velocities to compute the potential.
#         # TODO assume sampler_state.potential_energy is the correct potential if not None?
#         sampler_state.apply_to_context(context, ignore_velocities=True)
#         initial_energy = thermodynamic_state.reduced_potential(context)
#         logger.info(f"initial energy: {initial_energy}")

#         # Handle default and weird cases for atom_subset.
#         if self.atom_subset is None:
#             atom_subset = slice(None)
#         elif not isinstance(self.atom_subset, slice) and len(self.atom_subset) == 1:
#             # Slice so that initial_positions (below) will have a 2D shape.
#             atom_subset = slice(self.atom_subset[0], self.atom_subset[0]+1)
#         else:
#             atom_subset = self.atom_subset

#         # Store initial positions of the atoms that are moved.
#         # We'll use this also to recover in case the move is rejected.
#         if isinstance(atom_subset, slice):
#             # Numpy array when sliced return a view, they are not copied.
#             initial_positions = copy.deepcopy(sampler_state.positions[atom_subset])
#         else:
#             # This automatically creates a copy.
#             initial_positions = sampler_state.positions[atom_subset]

#         # Propose perturbed positions. Modifying the reference changes the sampler state.
#         proposed_positions = self._propose_positions(initial_positions)

#         # Compute the energy of the proposed positions.
#         sampler_state.positions[atom_subset] = proposed_positions
#         sampler_state.apply_to_context(context, ignore_velocities=True)
#         print("saving")
#         traj = md.Trajectory(np.array(sampler_state.positions), md.Topology.from_openmm(top))
#         traj.save("move.pdb")
#         proposed_energy = thermodynamic_state.reduced_potential(context)
#         print(f"proposed energy: {proposed_energy}")

#         # Accept or reject with Metropolis criteria.
#         delta_energy = proposed_energy - initial_energy
#         if (not np.isnan(proposed_energy) and
#                 (delta_energy <= 0.0 or np.random.rand() < np.exp(-delta_energy))):
#             self.n_accepted += 1
#         else:
#             # Restore original positions.
#             sampler_state.positions[atom_subset] = initial_positions
#         self.n_proposed += 1

#         # Print timing information.
#         timer.stop(benchmark_id)
#         #timer.report_timing()

        
#     @classmethod
#     def rotate_positions(cls, positions):
#         """Return the positions after applying a random rotation to them.
#         Parameters
#         ----------
#         positions : nx3 numpy.ndarray simtk.unit.Quantity
#             The positions to rotate.
#         Returns
#         -------
#         rotated_positions : nx3 numpy.ndarray simtk.unit.Quantity
#             The rotated positions.
#         """
#         positions_unit = positions.unit
#         x_initial = positions / positions_unit

#         # Compute center of geometry of atoms to rotate.
#         x_initial_mean = x_initial.mean(0)

#         # Generate a random rotation matrix.
#         rotation_matrix = cls.generate_random_rotation_matrix()

#         # Apply rotation.
#         x_proposed = (rotation_matrix * np.matrix(x_initial - x_initial_mean).T).T + x_initial_mean
#         print(unit.Quantity(x_proposed, positions_unit))
#         return unit.Quantity(x_proposed, positions_unit)

#     @classmethod
#     def generate_random_rotation_matrix(cls):
#         """Return a random 3x3 rotation matrix.
#         Returns
#         -------
#         Rq : 3x3 numpy.ndarray
#             The random rotation matrix.
#         """
#         q = cls._generate_uniform_quaternion()
#         return cls._rotation_matrix_from_quaternion(q)

#     @staticmethod
#     def _rotation_matrix_from_quaternion(q):
#         """Compute a 3x3 rotation matrix from a given quaternion (4-vector).
#         Parameters
#         ----------
#         q : 1x4 numpy.ndarray
#             Quaterion (need not be normalized, zero norm OK).
#         Returns
#         -------
#         Rq : 3x3 numpy.ndarray
#             Orthogonal rotation matrix corresponding to quaternion q.
#         Examples
#         --------
#         >>> q = np.array([0.1, 0.2, 0.3, -0.4])
#         >>> Rq = MCRotationMove._rotation_matrix_from_quaternion(q)
#         References
#         ----------
#         [1] http://en.wikipedia.org/wiki/Rotation_matrix#Quaternion
#         """

#         w, x, y, z = q
#         Nq = (q**2).sum()  # Squared norm.
#         if Nq > 0.0:
#             s = 2.0 / Nq
#         else:
#             s = 0.0

#         X = x*s;   Y = y*s;  Z = z*s
#         wX = w*X; wY = w*Y; wZ = w*Z
#         xX = x*X; xY = x*Y; xZ = x*Z
#         yY = y*Y; yZ = y*Z; zZ = z*Z

#         Rq = np.matrix([[1.0-(yY+zZ),     xY-wZ,          xZ+wY],
#                         [xY+wZ,        1.0-(xX+zZ),       yZ-wX],
#                         [xZ-wY,           yZ+wX,    1.0-(xX+yY)]])

#         return Rq
    
    
#     def _rotation_matrix(self, axis, theta):
#         """Function returns the rotation matrix associated with counterclockwise rotation
#         about the given axis by theta radians.
#         Parameters
#         ----------
#         axis :
#         theta : float
#             The angle of rotation in radians.
#         """
#         axis = np.asarray(axis)
#         axis = axis / math.sqrt(np.dot(axis, axis))
#         a = math.cos(theta / 2.0)
#         b, c, d = -axis * math.sin(theta / 2.0)
#         aa, bb, cc, dd = a * a, b * b, c * c, d * d
#         bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
#         return np.array([[aa + bb - cc - dd, 2 * (bc + ad),
#                              2 * (bd - ac)], [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
#                             [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
    
#     @staticmethod
#     def _generate_uniform_quaternion():
#         """Generate a uniform normalized quaternion 4-vector.
#         References
#         ----------
#         [1] K. Shoemake. Uniform random rotations. In D. Kirk, editor,
#         Graphics Gems III, pages 124-132. Academic, New York, 1992.
#         [2] Described briefly here: http://planning.cs.uiuc.edu/node198.html
#         Examples
#         --------
#         >>> q = MCRotationMove._generate_uniform_quaternion()
#         """
#         u = np.random.rand(3)
#         q = np.array([np.sqrt(1-u[0])*np.sin(2*np.pi*u[1]),
#                       np.sqrt(1-u[0])*np.cos(2*np.pi*u[1]),
#                       np.sqrt(u[0])*np.sin(2*np.pi*u[2]),
#                       np.sqrt(u[0])*np.cos(2*np.pi*u[2])])
#         return q

# #     def _propose_positions(self, initial_positions):
# #         """Implement MetropolizedMove._propose_positions for apply()"""
# #         return self.rotate_positions(initial_positions)

#     def _propose_positions(self, initial_positions): # assumes initial_positions is for all atoms
        
# #         # Create openeye molecule
# #         molecule = oeommtools.openmmTop_to_oemol(self.topology, initial_positions, verbose=False)
# #         oechem.OEPerceiveResidues(molecule)
# #         oechem.OEFindRingAtomsAndBonds(molecule)

#         # Get backbone atom indices
#         backbone_indices = md.Topology.from_openmm(self.topology).select("backbone")
        
# #         # Get sidechain OpenMM atom indices
# #         sidechain_indices = []
# #         for atom in self.topology.atoms():
# #             if atom.index not in backbone_indices and atom.residue.chain.index == self.chain_index and atom.residue.index == self.residue_index:
# #                 sidechain_indices.append(atom.index)
        
# #        # Get rotatable bonds
# #         rotatable_bonds = []
# #         for atom in molecule.GetAtoms():
# #             if atom.GetIdx() in sidechain_indices:
# #                 for bond in atom.GetBonds():
# #                     start_atom = bond.GetBgn()
# #                     end_atom = bond.GetEnd()
# #                     if start_atom.GetAtomicNum() > 1 and end_atom.GetAtomicNum() > 1 and bond.IsRotor():
# #                         if bond not in rotatable_bonds:
# #                             rotatable_bonds.append(bond)
#         # Get rotatable bond
#         rotatable_bond = None
#         for bond in self.topology.bonds():
#             if bond[0].index == self.bond_indices[0] and bond[1].index == self.bond_indices[1]:
#                 rotatable_bond = bond
#         if not rotatable_bond:
#             raise Exception("Specified bond not found!")
        
#         # Get atoms to be rotated
#         rotatable_atoms = [rotatable_bond[0], rotatable_bond[1]]
#         for atom in rotatable_bond:
#             if atom.index not in backbone_indices:
#                 # Get neighbors of the rotatable bond's atoms
#                 neighbors = []
#                 for bond in self.topology.bonds():
#                     if bond[0].index == atom.index:
#                         neighbors.append(bond[1])
#                     elif bond[1].index == atom.index:
#                         neighbors.append(bond[0])
#                 # Add neighbors and neighbors of neighbors to rotatable_atoms
#                 for neighbor in neighbors:
#                     if neighbor not in rotatable_atoms and neighbor.index not in backbone_indices:
#                         rotatable_atoms.append(neighbor)
#                         if neighbor.element.atomic_number > 1:
#                             # Get neighbors of neighbors
#                             neighbors_secondary = []
#                             for bond in self.topology.bonds():
#                                 if bond[0].index == neighbor.index:
#                                     neighbors_secondary.append(bond[1])
#                                 elif bond[1].index == neighbor.index:
#                                     neighbors_secondary.append(bond[0])
#                             for neighbor_secondary in neighbors_secondary:
#                                 if neighbor_secondary not in rotatable_atoms and neighbor_secondary.index not in backbone_indices:
#                                     rotatable_atoms.append(neighbor_secondary)
        
# #         print(backbone_indices)
# #         print(rotatable_bond, rotatable_bond[0], rotatable_bond[1])
# #         print(rotatable_atoms)
        
#         theta = random.random() * 2 * math.pi
        
#         print(f"Rotating bond: {rotatable_bond} in resnum: {self.residue_index} by {theta} radians")
#         print(initial_positions)
# #         model = copy.copy(self.structure)
# #         atoms = list(model.topology.atoms())
#         new_positions = copy.deepcopy(initial_positions)
    
#         # Find the rotation axis using the initial positions
#         axis1 = rotatable_atoms[0].index
#         axis2 = rotatable_atoms[1].index
#         rotation_axis = (initial_positions[axis1] - initial_positions[axis2]) / initial_positions.unit
#         print(f"rotation axis: {rotation_axis}")
        
#         # Calculate the rotation matrix
#         rotation_matrix = self._rotation_matrix(rotation_axis, theta)
#         print(f"rotation matrix: {rotation_matrix}")

#         # Apply the rotation matrix to the target atoms
#         for atom in rotatable_atoms:
#             # Find the reduced position (substract out axis)
#             reduced_position = (initial_positions[atom.index] - initial_positions[axis2])._value
            
#             # Find the new positions by multiplying by rot matrix
#             new_position = np.dot(rotation_matrix, reduced_position) * initial_positions.unit + initial_positions[axis2]

#             # Update the new positions
#             new_positions[atom.index][0] = new_position[0] 
#             new_positions[atom.index][1] = new_position[1] 
#             new_positions[atom.index][2] = new_position[2]

#         print(new_positions)
        
#         return new_positions

        


And here is the cleaned up version, with minimal changes to the existing MetropolizedMove class

In [208]:
# class MCDihedralRotationMove(mcmc.MetropolizedMove):
#     """A metropolized move that randomly rotates a subset of atoms about a specified bond.
    
#     Parameters
#     ----------
#     atom_subset : slice or list of int
#         Atom indices where the first four are the dihedral of interest. The rotatable bond is specified by
#         the 2nd and 3rd atom indices in the list. All atom indices after the 3rd correspond to the atoms to be rotated.
#         Example: [6, 8, 10, 18, 13, 14, 15, 16, 17, 19] means that the dihedral of interest is 6-8-10-18,
#         the rotatable bond is 8-10, and the atoms to be rotated are 18, 13, 14, 15, 16, 17, 19.
#     theta : float, default=None
#         Desired angle to rotate to. If not specified, theta will be chosen randomly.
#         Must be between -pi and pi.
#         Example: If the current angle is pi, and theta is 0, the atoms will be rotated by -pi.
#     context_cache : openmmtools.cache.ContextCache, optional
#         The ContextCache to use for Context creation. If None, the global cache
#         openmmtools.cache.global_context_cache is used (default is None).
    
#     Attributes
#     ----------
#     atom_subset
#     theta
#     context_cache
#     n_accepted : int
#         The number of proposals accepted.
#     n_proposed : int
#         The total number of attempted moves.
#     proposed_positions : np.array of floats
#         Proposed positions of atoms in atoms_subset
    
#     See Also
#     --------
#     MetropolizedMove
#     """

#     def __init__(self, atom_subset, theta=-np.inf, context_cache=None):
#         self.atom_subset = atom_subset
#         self.theta = theta
#         self.context_cache = context_cache
#         self.n_accepted = 0
#         self.n_proposed = 0
#         self.proposed_positions = None
        
#     def apply(self, thermodynamic_state, sampler_state):
#         """Apply a metropolized move to the sampler state.
#         Total number of acceptances and proposed move are updated.
#         Parameters
#         ----------
#         thermodynamic_state : openmmtools.states.ThermodynamicState
#            The thermodynamic state to use to apply the move.
#         sampler_state : openmmtools.states.SamplerState
#            The initial sampler state to apply the move to. This is modified.
#         """
#         timer = Timer()
#         benchmark_id = 'Applying {}'.format(self.__class__.__name__ )
#         timer.start(benchmark_id)

#         # Check if we have to use the global cache.
#         if self.context_cache is None:
#             context_cache = cache.global_context_cache
#         else:
#             context_cache = self.context_cache

#         # Create context, any integrator works.
#         context, unused_integrator = context_cache.get_context(thermodynamic_state)

#         # Compute initial energy. We don't need to set velocities to compute the potential.
#         # TODO assume sampler_state.potential_energy is the correct potential if not None?
#         sampler_state.apply_to_context(context, ignore_velocities=True)
#         initial_energy = thermodynamic_state.reduced_potential(context)
#         print(f"initial_energy: {initial_energy}")
        
#         # Handle default and weird cases for atom_subset.
#         if self.atom_subset is None:
#             atom_subset = slice(None)
#         elif not isinstance(self.atom_subset, slice) and len(self.atom_subset) == 1:
#             # Slice so that initial_positions (below) will have a 2D shape.
#             atom_subset = slice(self.atom_subset[0], self.atom_subset[0]+1)
#         else:
#             atom_subset = self.atom_subset

#         # Store initial positions of the atoms that are moved.
#         # We'll use this also to recover in case the move is rejected.
#         if isinstance(atom_subset, slice):
#             # Numpy array when sliced return a view, they are not copied.
#             initial_positions = copy.deepcopy(sampler_state.positions[atom_subset])
#         else:
#             # This automatically creates a copy.
#             initial_positions = sampler_state.positions[atom_subset]

#         import copy
#         from simtk import openmm
#         from openmmtools.constants import kB
#         temperature = 300.0 * unit.kelvin
#         kT = kB * temperature
#         beta = 1.0/kT
#         platform = openmm.Platform.getPlatformByName('CUDA')
#         platform.setPropertyDefaultValue('CudaPrecision', 'mixed')
#         platform.setPropertyDefaultValue('DeterministicForces', 'true')
#         system = context.getSystem()
#         system = copy.deepcopy(system)
#         # Get positions.
#         positions = context.getState(getPositions=True).getPositions(asNumpy=True)
#         # Get Parameters
#         parameters = context.getParameters()
#         # Segregate forces.
#         for index in range(system.getNumForces()):
#             force = system.getForce(index)
#             force.setForceGroup(index)
#         # Create new Context.
#         integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
#         context2 = openmm.Context(system, integrator, platform)
#         context2.setPositions(positions)
#         for (parameter, value) in parameters.items():
#             context2.setParameter(parameter, value)
#         energy_components = list()
#         for index in range(system.getNumForces()):
#             force = system.getForce(index)
#             forcename = force.__class__.__name__
#             groups = 1<<index
#             potential = beta * context2.getState(getEnergy=True, groups=groups).getPotentialEnergy()
#             energy_components.append((forcename, potential))
#         del context2, integrator
#         print(energy_components)
            
#         # Propose perturbed positions. Modifying the reference changes the sampler state.
#         proposed_positions = self._propose_positions(initial_positions)

#         # Compute the energy of the proposed positions.
#         sampler_state.positions[atom_subset] = proposed_positions
#         sampler_state.apply_to_context(context, ignore_velocities=True)
#         proposed_energy = thermodynamic_state.reduced_potential(context)
#         print(f"proposed_energy: {proposed_energy}")
        
#         import copy
#         from simtk import openmm
#         from openmmtools.constants import kB
#         temperature = 300.0 * unit.kelvin
#         kT = kB * temperature
#         beta = 1.0/kT
#         platform = openmm.Platform.getPlatformByName('CUDA')
#         platform.setPropertyDefaultValue('CudaPrecision', 'mixed')
#         platform.setPropertyDefaultValue('DeterministicForces', 'true')
#         system = context.getSystem()
#         system = copy.deepcopy(system)
#         # Get positions.
#         positions = context.getState(getPositions=True).getPositions(asNumpy=True)
#         # Get Parameters
#         parameters = context.getParameters()
#         # Segregate forces.
#         for index in range(system.getNumForces()):
#             force = system.getForce(index)
#             force.setForceGroup(index)
#         # Create new Context.
#         integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
#         context2 = openmm.Context(system, integrator, platform)
#         context2.setPositions(positions)
#         for (parameter, value) in parameters.items():
#             context2.setParameter(parameter, value)
#         energy_components = list()
#         for index in range(system.getNumForces()):
#             force = system.getForce(index)
#             forcename = force.__class__.__name__
#             groups = 1<<index
#             potential = beta * context2.getState(getEnergy=True, groups=groups).getPotentialEnergy()
#             energy_components.append((forcename, potential))
#         del context2, integrator
#         print(energy_components)
        
        
#         # Accept or reject with Metropolis criteria.
#         delta_energy = proposed_energy - initial_energy
#         if (not np.isnan(proposed_energy) and
#                 (delta_energy <= 0.0 or np.random.rand() < np.exp(-delta_energy))):
#             self.n_accepted += 1
#         else:
#             # Restore original positions.
#             sampler_state.positions[atom_subset] = initial_positions
#         self.n_proposed += 1

#         # Print timing information.
#         timer.stop(benchmark_id)
#         #timer.report_timing()
    
#     def generate_rotation_matrix(self, axis):
#         """Generate the rotation matrix associated with counterclockwise rotation
#         about the given axis by theta radians.
        
#         Parameters
#         ----------
#         axis : np.array of three floats
#             The axis of rotation (unitless)
        
#         Returns
#         -------
#         np.array of floats
#             Rotation matrix
        
#         """
#         axis = np.asarray(axis)
#         axis = axis / math.sqrt(np.dot(axis, axis))
#         a = math.cos(self.theta / 2.0)
#         b, c, d = -axis * math.sin(self.theta / 2.0)
#         aa, bb, cc, dd = a * a, b * b, c * c, d * d
#         bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
#         return np.array([[aa + bb - cc - dd, 2 * (bc + ad),
#                              2 * (bd - ac)], [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
#                             [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])
    
    
#     @classmethod
#     def compute_dihedral(cls, dihedral):
#         """Given the positions of four atoms that for a dihedral, compute the angle in radians.
        
#         Parameters
#         ----------
#         dihedral : np.array or list of floats
#             Positions of atoms forming the dihedral
        
#         Returns
#         -------
#         float
#             Dihedral angle in radians
        
#         """    
    
#         def _cross_vec3(a, b):
#             c = np.zeros(3)
#             c[0] = a[1]*b[2] - a[2]*b[1]
#             c[1] = a[2]*b[0] - a[0]*b[2]
#             c[2] = a[0]*b[1] - a[1]*b[0]
#             return c

#         def _norm(a):
#             n_2 = np.dot(a, a)
#             return np.sqrt(n_2)
        
#         atom_position = dihedral[0] / dihedral[0].unit
#         bond_position = dihedral[1] / dihedral[1].unit
#         angle_position = dihedral[2] / dihedral[2].unit
#         torsion_position = dihedral[3] / dihedral[3].unit
        
#         a = atom_position - bond_position
#         b = angle_position - bond_position
#         c = angle_position - torsion_position
#         a_u = a / _norm(a)
#         b_u = b / _norm(b)
#         c_u = c / _norm(c)
        
#         plane1 = _cross_vec3(a_u, b_u)
#         plane2 = _cross_vec3(b_u, c_u)

#         cos_phi = np.dot(plane1, plane2) / (_norm(plane1)*_norm(plane2))
#         if cos_phi < -1.0:
#             cos_phi = -1.0
#         elif cos_phi > 1.0:
#             cos_phi = 1.0

#         phi = np.arccos(cos_phi)

#         if np.dot(a, plane2) <= 0:
#             phi = -phi

#         return phi

#     def rotate_positions(self, initial_positions):
#         """Apply random rotation to positions.
        
#         Parameters
#         ----------
#         initial_positions : numpy.ndarray simtk.unit.Quantity
#             The positions of all atoms in atom_subset.
#         rotatable_atoms : list of app.Topology.Atom
#             Atoms to be rotated
#         rotatable_bond : app.Topology.Bond
#             Bond to be used as the axis of rotation

        
#         Returns
#         -------
#         rotated_positions : numpy.ndarray simtk.unit.Quantity
#             The rotated positions.
#         """
        
#         old_angle = self.compute_dihedral(initial_positions[:4])
#         print("old angle: ", old_angle)
#         if self.theta == -np.inf:
#             # Choose a random rotation angle
#             self.theta = random.uniform(-math.pi, math.pi)
#         else:
#             # Use specified angle
#             print("theta ", self.theta)
#             if not (self.theta < math.pi and self.theta > -math.pi):
#                 raise Exception("Theta must be less than pi and greater than -pi")
#             self.theta = self.theta - old_angle
#         logger.info(f"Rotating by {self.theta} radians")
        
#         # Make a copy of the initial positions 
#         new_positions = copy.deepcopy(initial_positions)
    
#         # Find the rotation axis using the initial positions
#         axis1 = 1
#         axis2 = 2
#         rotation_axis = (initial_positions[axis1] - initial_positions[axis2]) / initial_positions.unit
        
#         # Calculate the rotation matrix
#         rotation_matrix = self.generate_rotation_matrix(rotation_axis)

#         # Apply the rotation matrix to the target atoms
#         for atom_index in range(3, len(self.atom_subset)):
#             # Find the reduced position (substract out axis)
#             reduced_position = (initial_positions[atom_index] - initial_positions[axis2])._value
            
#             # Find the new positions by multiplying by rot matrix
#             new_position = np.dot(rotation_matrix, reduced_position) * initial_positions.unit + initial_positions[axis2]

#             # Update the new positions
#             new_positions[atom_index][0] = new_position[0] 
#             new_positions[atom_index][1] = new_position[1] 
#             new_positions[atom_index][2] = new_position[2]
        
#         return new_positions

        
#     def _propose_positions(self, initial_positions):
#         """Implement MetropolizedMove._propose_positions for apply()"""
#         self.proposed_positions = self.rotate_positions(initial_positions)
#         return self.proposed_positions


In [169]:

class MCDihedralRotationMove(mcmc.MetropolizedMove):
    """A metropolized move that randomly rotates a subset of atoms about a specified bond.

    Parameters
    ----------
    atom_subset : slice or list of int
        Atom indices where the first four are the dihedral of interest. The rotatable bond is specified by
        the 2nd and 3rd atom indices in the list. All atom indices after the 3rd correspond to the atoms to be rotated.
        Example: [6, 8, 10, 18, 13, 14, 15, 16, 17, 19] means that the dihedral of interest is 6-8-10-18,
        the rotatable bond is 8-10, and the atoms to be rotated are 18, 13, 14, 15, 16, 17, 19.
    desired_angle : float, default -np.inf
        Desired angle to rotate to. If not specified, this will be chosen randomly. Must be between -pi and pi.
        Example: If the current angle is pi, and desired_angle is 0, the atoms will be rotated by -pi.
    context_cache : openmmtools.cache.ContextCache, optional
        The ContextCache to use for Context creation. If None, the global cache
        openmmtools.cache.global_context_cache is used (default is None).

    Attributes
    ----------
    atom_subset
    desired_angle
    context_cache
    n_accepted : int
        The number of proposals accepted.
    n_proposed : int
        The total number of attempted moves.
    proposed_positions : np.array of floats
        Proposed positions of atoms in atom_subset

    See Also
    --------
    MetropolizedMove
    """

    def __init__(self, atom_subset, desired_angle=-np.inf, context_cache=None):
        self.atom_subset = atom_subset
        self.desired_angle = desired_angle
        self.context_cache = context_cache
        self.n_accepted = 0
        self.n_proposed = 0
        self.proposed_positions = None
    
#     def apply(self, thermodynamic_state, sampler_state):
#         """Apply a metropolized move to the sampler state.
#         Total number of acceptances and proposed move are updated.
#         Parameters
#         ----------
#         thermodynamic_state : openmmtools.states.ThermodynamicState
#            The thermodynamic state to use to apply the move.
#         sampler_state : openmmtools.states.SamplerState
#            The initial sampler state to apply the move to. This is modified.
#         """
#         timer = Timer()
#         benchmark_id = 'Applying {}'.format(self.__class__.__name__ )
#         timer.start(benchmark_id)

#         # Check if we have to use the global cache.
#         if self.context_cache is None:
#             context_cache = cache.global_context_cache
#         else:
#             context_cache = self.context_cache

#         # Create context, any integrator works.
#         context, unused_integrator = context_cache.get_context(thermodynamic_state)
#         print("before_context: ", sampler_state.positions)
#         # Compute initial energy. We don't need to set velocities to compute the potential.
#         # TODO assume sampler_state.potential_energy is the correct potential if not None?
#         sampler_state.apply_to_context(context, ignore_velocities=True)
#         initial_energy = thermodynamic_state.reduced_potential(context)

#         # Handle default and weird cases for atom_subset.
#         if self.atom_subset is None:
#             atom_subset = slice(None)
#         elif not isinstance(self.atom_subset, slice) and len(self.atom_subset) == 1:
#             # Slice so that initial_positions (below) will have a 2D shape.
#             atom_subset = slice(self.atom_subset[0], self.atom_subset[0]+1)
#         else:
#             atom_subset = self.atom_subset
#         print(f"atom_subset: {atom_subset}")
#         print(sampler_state.positions)
#         # Store initial positions of the atoms that are moved.
#         # We'll use this also to recover in case the move is rejected.
#         if isinstance(atom_subset, slice):
#             # Numpy array when sliced return a view, they are not copied.
#             initial_positions = copy.deepcopy(sampler_state.positions[atom_subset])
#             print("is a slice")
#         else:
#             # This automatically creates a copy.
#             initial_positions = sampler_state.positions[atom_subset]
#             print("is not a slice")
#         print(f"initial pos: {initial_positions}")
#         # Propose perturbed positions. Modifying the reference changes the sampler state.
#         proposed_positions = self._propose_positions(initial_positions)

#         # Compute the energy of the proposed positions.
#         sampler_state.positions[atom_subset] = proposed_positions
#         sampler_state.apply_to_context(context, ignore_velocities=True)
#         proposed_energy = thermodynamic_state.reduced_potential(context)

#         # Accept or reject with Metropolis criteria.
#         delta_energy = proposed_energy - initial_energy
#         if (not np.isnan(proposed_energy) and
#                 (delta_energy <= 0.0 or np.random.rand() < np.exp(-delta_energy))):
#             self.n_accepted += 1
#         else:
#             # Restore original positions.
#             sampler_state.positions[atom_subset] = initial_positions
#         self.n_proposed += 1

#         # Print timing information.
#         timer.stop(benchmark_id)
#         #timer.report_timing()
    
    @staticmethod
    def generate_rotation_matrix(axis, theta):
        """Generate the rotation matrix associated with counterclockwise rotation
        about the given axis by theta radians.

        Parameters
        ----------
        axis : np.array of three floats
            The axis of rotation (unitless)
        theta : float
            Rotation angle in radians

        Returns
        -------
        np.array of floats
            Rotation matrix

        """
        axis = np.asarray(axis)
        axis = axis / math.sqrt(np.dot(axis, axis))
        a = math.cos(theta / 2.0)
        b, c, d = -axis * math.sin(theta / 2.0)
        aa, bb, cc, dd = a * a, b * b, c * c, d * d
        bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
        return np.array([[aa + bb - cc - dd, 2 * (bc + ad),
                          2 * (bd - ac)], [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                         [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

    @staticmethod
    def compute_dihedral(dihedral):
        """Given the positions of four atoms that form a dihedral, compute the angle in radians.

        Parameters
        ----------
        dihedral : np.array or list of floats
            Positions of atoms forming the dihedral

        Returns
        -------
        float
            Dihedral angle in radians

        """

        def _cross_vec3(a, b):
            c = np.zeros(3)
            c[0] = a[1] * b[2] - a[2] * b[1]
            c[1] = a[2] * b[0] - a[0] * b[2]
            c[2] = a[0] * b[1] - a[1] * b[0]
            return c

        def _norm(a):
            n_2 = np.dot(a, a)
            return np.sqrt(n_2)

        atom_position = dihedral[0] / dihedral[0].unit
        bond_position = dihedral[1] / dihedral[1].unit
        angle_position = dihedral[2] / dihedral[2].unit
        torsion_position = dihedral[3] / dihedral[3].unit

        a = atom_position - bond_position
        b = angle_position - bond_position
        c = angle_position - torsion_position
        a_u = a / _norm(a)
        b_u = b / _norm(b)
        c_u = c / _norm(c)

        plane1 = _cross_vec3(a_u, b_u)
        plane2 = _cross_vec3(b_u, c_u)

        cos_phi = np.dot(plane1, plane2) / (_norm(plane1) * _norm(plane2))
        if cos_phi < -1.0:
            cos_phi = -1.0
        elif cos_phi > 1.0:
            cos_phi = 1.0

        phi = np.arccos(cos_phi)

        if np.dot(a, plane2) <= 0:
            phi = -phi

        return phi

    def rotate_positions(self, initial_positions):
        """Apply rotation to atom_subset positions.

        Parameters
        ----------
        initial_positions : numpy.ndarray simtk.unit.Quantity
            The positions of all atoms in atom_subset.

        Returns
        -------
        rotated_positions : numpy.ndarray simtk.unit.Quantity
            The rotated positions.
        """

        old_angle = self.compute_dihedral(initial_positions[:4])

        if self.desired_angle == -np.inf:
            # Choose a random rotation angle
            theta = random.uniform(-math.pi, math.pi)
        else:
            # If desired_angle is specified, determine rotation angle
            if not (self.desired_angle <= math.pi and self.desired_angle >= -math.pi):
                raise Exception("Desired angle must be less than pi and greater than -pi")
            theta = self.desired_angle - old_angle
        print(old_angle)
        print(f"Rotating by {theta} radians")

        # Make a copy of the initial positions
        new_positions = copy.deepcopy(initial_positions)

        # Find the rotation axis using the initial positions
        axis1 = 1
        axis2 = 2
        rotation_axis = (initial_positions[axis1] - initial_positions[axis2]) / initial_positions.unit
        print("rotation axis ", rotation_axis)

        # Calculate the rotation matrix
        rotation_matrix = self.generate_rotation_matrix(rotation_axis, theta)
        print("rotation matrix ", rotation_matrix)

        # Apply the rotation matrix to the target atoms
        for atom_index in range(3, len(self.atom_subset)):
            # Find the reduced position (substract out axis)
            reduced_position = (initial_positions[atom_index] - initial_positions[axis2])._value

            # Find the new positions by multiplying by rot matrix
            new_position = np.dot(rotation_matrix, reduced_position) * initial_positions.unit + initial_positions[axis2]

            # Update the new positions
            new_positions[atom_index][0] = new_position[0]
            new_positions[atom_index][1] = new_position[1]
            new_positions[atom_index][2] = new_position[2]
        print("new pos ", new_positions)
        return new_positions

    def _propose_positions(self, initial_positions):
        """Implement MetropolizedMove._propose_positions for apply()"""
        self.proposed_positions = self.rotate_positions(initial_positions)
        return self.proposed_positions

In [170]:
# move = MCDihedralRotationMove([6, 8, 10, 18, 13, 14, 15, 16, 17, 19], desired_angle=0) # THR
move = MCDihedralRotationMove([6, 8, 10, 11, 12, 13], desired_angle=-0.9554943789) # Alanine dipeptide
# move = MCDihedralRotationMove([6, 8, 10, 11, 12, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]) # TYR
# move = MCDihedralRotationMove([8, 12, 15, 17, 43, 19, 45, 21, 51, 21, 23, 48, 25, 50], desired_angle=1.2047) # Thrombin
# move = MCDihedralRotationMove([6, 8, 10, 18, 13, 14, 15, 16, 17, 19, 26, 27]) # Hybrid system (THR)


In [171]:
sampler = mcmc.MCMCSampler(thermodynamic_state, sampler_state, move=move)


In [161]:
sampler.minimize()


In [162]:
traj = md.Trajectory(np.array(sampler.sampler_state.positions), md.Topology.from_openmm(top))
# traj = md.Trajectory(np.array(sampler.sampler_state.positions), md.Topology.from_openmm(topology))
# traj = md.Trajectory(np.array(htf.old_positions(sampler.sampler_state.positions)), md.Topology.from_openmm(htf._topology_proposal.old_topology))


In [163]:
traj.save("before.pdb")


In [172]:
pos = list()
for i in range(1):
    sampler.run(n_iterations=1)
    pos.append(np.array(sampler.sampler_state.positions))
    

1.047198822205612
Rotating by -2.002693201105612 radians
rotation axis  [-0.08080423  0.03930828  0.12321437]
rotation matrix  [[-0.02031582  0.54002284 -0.84140513]
 [-0.9275184  -0.32434309 -0.18577185]
 [-0.37322498  0.77664463  0.50747042]]
new pos  [[ 3.55537540e-01  3.96964880e-01 -3.10000000e-07]
 [ 4.85326210e-01  4.61392530e-01 -4.30000000e-07]
 [ 5.66130440e-01  4.22084250e-01 -1.23214800e-01]
 [ 6.59099143e-01  4.78943429e-01 -1.25418242e-01]
 [ 5.88841910e-01  3.15546115e-01 -1.19365331e-01]
 [ 5.08287672e-01  4.43627550e-01 -2.13054053e-01]] nm


In [88]:
for position in test.positions:
    print(position)

[ 2.000001e-01  1.000000e-01 -1.300000e-07] nm
[2.000001e-01 2.090000e-01 1.000000e-08] nm
[0.1486264 0.2453849 0.0889824] nm
[ 0.1486259  0.2453852 -0.088982 ] nm
[ 3.427420e-01  2.640795e-01 -3.000000e-07] nm
[ 4.390580e-01  1.877406e-01 -6.600000e-07] nm
[ 3.5553754e-01  3.9696488e-01 -3.1000000e-07] nm
[ 2.7331200e-01  4.5561601e-01 -1.3000000e-07] nm
[ 4.8532621e-01  4.6139253e-01 -4.3000000e-07] nm
[0.5407596  0.43155388 0.08898152] nm
[ 0.56613044  0.42208425 -0.1232148 ] nm
[ 0.51232615  0.4521363  -0.21312016] nm
[ 0.6630484   0.47189354 -0.12057907] nm
[ 0.58085401  0.31408724 -0.1241385 ] nm
[4.7126759e-01 6.1294185e-01 1.4000000e-07] nm
[3.6006445e-01 6.6527027e-01 6.2000000e-07] nm
[5.8460533e-01 6.8348833e-01 2.5000000e-07] nm
[ 6.7370014e-01  6.3591620e-01 -4.0000000e-08] nm
[5.8460551e-01 8.2838837e-01 6.2000000e-07] nm
[4.8185761e-01 8.6477349e-01 1.0400000e-06] nm
[0.63597984 0.86477313 0.08898283] nm
[ 0.635979    0.86477354 -0.08898187] nm


In [70]:
np.allclose(1, 1.0000003)

True

In [64]:
# pos[0][[6, 8, 10, 18, 13, 14, 15, 16, 17, 19]] = sampler.move.proposed_positions
pos[0][[6, 8, 10, 11, 12, 13]] = sampler.move.proposed_positions
# pos[0][[6, 8, 10, 11, 12, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]] = sampler.move.proposed_positions
# pos[0][[8, 12, 15, 17, 43, 19, 45, 21, 51, 21, 23, 48, 25, 50]] = sampler.move.proposed_positions
# pos[0][[6, 8, 10, 18, 13, 14, 15, 16, 17, 19, 26, 27]] = sampler.move.proposed_positions


In [65]:
traj = md.Trajectory(np.array(pos[0]), md.Topology.from_openmm(top))
# traj = md.Trajectory(np.array(pos[0]), md.Topology.from_openmm(topology))
# traj = md.Trajectory(np.array(htf.old_positions(pos[0] * unit.nanometer)), md.Topology.from_openmm(htf._topology_proposal.old_topology))


In [66]:
traj.save("after.pdb")

In [67]:
sampler.move.n_accepted

1